In [1]:
import pandas as pd 
import datetime
from shapely import wkt, Point
from tqdm import tqdm

<h3>For regular staypoints.csv</h3>

In [2]:
tripleg = pd.read_csv('triplegs.csv')
stop_df = pd.read_csv('staypoints.csv')

<h3>The end location and time is start time of next stop point.
</h3>

In [3]:
stop_df['started_at'] = pd.to_datetime(stop_df['started_at'])
tripleg['finished_at'] = pd.to_datetime(tripleg['finished_at'])

In [4]:
len(stop_df)

311572

In [5]:
ID_list=stop_df['user_id'].unique()
len(ID_list)

10096

In [6]:
tripleg

,id,user_id,started_at,finished_at,geom
0,1,1,2022-09-06 13:11:03-04:00,2022-09-06 13:11:54-04:00,LINESTRING (-82.0707699999999960 29.1930100000...
1,5,1,2022-09-09 00:59:39-04:00,2022-09-09 00:59:49-04:00,LINESTRING (-82.1166076699999934 29.1995639800...
2,6,2,2022-09-06 09:15:08-04:00,2022-09-06 09:15:58-04:00,LINESTRING (-81.6775345799999997 29.6550393099...
3,7,2,2022-09-06 21:34:07-04:00,2022-09-06 21:34:57-04:00,LINESTRING (-81.6840577100000047 29.6517777400...
4,8,2,2022-09-07 00:05:48-04:00,2022-09-07 00:09:57-04:00,LINESTRING (-81.6582870000000014 29.6534939999...
...,...,...,...,...,...
797312,837979,34494,2022-10-27 01:06:08-04:00,2022-10-27 01:09:53-04:00,LINESTRING (-81.6119800000000026 30.3824900000...
797313,837980,34496,2022-10-31 02:06:17-04:00,2022-10-31 02:07:50-04:00,LINESTRING (-81.7572399999999959 30.3360599999...
797314,837981,34497,2022-10-26 08:25:34-04:00,2022-10-26 08:25:48-04:00,LINESTRING (-82.0980799999999959 29.9914400000...
797315,837982,34497,2022-10-29 20:39:33-04:00,2022-10-29 20:39:40-04:00,LINESTRING (-82.2332099999999997 29.3812399999...


In [7]:
time_window = datetime.timedelta(minutes=5)

miss = 0
check_len = 0
data_to_append_list = []
for id in tqdm(ID_list):
    stop_by_user = stop_df[stop_df['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for stop_row in stop_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > stop_row[3] - time_window) &
            (related_tripled_by_user['finished_at'] <= stop_row[3])
        ]
        if len(tripled_found) == 0: continue #did not found related trip
        #print(tripled_found)
        #check if there if repeat data
        if len(tripled_found) > 1: 
            check_len += 1
            tripled_found = tripled_found.nlargest(1, 'finished_at')
        
        '''
            The end location and time is start time of next stop point.
        '''
        
        # data_to_append = {'deviceID': id, 'tripleg_ID': tripled_found.iloc[0]['id'], 
        #                   'trip_start_location': Point(list(wkt.loads(tripled_found.iloc[0]['geom']).coords)[0]),
        #                   'trip_end_location': stop_row[5],
        #                   'trip_start_timestamp': tripled_found.iloc[0]['started_at'],
        #                   'trip_end_timestamp': stop_row[3]}
        data_to_append = {'user_id': id, 'tripleg_ID': tripled_found.iloc[0]['id'], 
                          'trip_started_at': tripled_found.iloc[0]['started_at'],
                          'trip_finished_at': tripled_found.iloc[0]['finished_at'],
                          'trip':tripled_found.iloc[0]['geom']}
        
        data_to_append_list.append(data_to_append)
        

trip_df = pd.DataFrame(data_to_append_list, columns=['user_id', 'tripleg_ID', 'trip_started_at', 'trip_finished_at', 'trip'])


100%|██████████| 10096/10096 [02:57<00:00, 56.93it/s] 


In [8]:
trip_df

,user_id,tripleg_ID,trip_started_at,trip_finished_at,trip
0,10,758,2022-10-07 22:00:05-04:00,2022-10-07 22:00:34-04:00,LINESTRING (-81.6504979999999989 30.4700999999...
1,10,764,2022-10-08 13:45:17-04:00,2022-10-08 13:45:27-04:00,LINESTRING (-81.6341471700000056 30.4783272700...
2,10,770,2022-10-09 13:00:02-04:00,2022-10-09 13:00:51-04:00,LINESTRING (-81.6210899999999953 30.4746300000...
3,10,797,2022-10-11 22:00:32-04:00,2022-10-11 22:00:46-04:00,LINESTRING (-81.6504999999999939 30.4700999999...
4,14,930,2022-09-11 16:42:15-04:00,2022-09-11 16:42:40-04:00,LINESTRING (-82.6358900000000034 29.4446900000...
...,...,...,...,...,...
40277,34471,837931,2022-10-25 20:03:34-04:00,2022-10-25 20:10:21-04:00,LINESTRING (-82.8051399999999944 30.3151800000...
40278,34471,837932,2022-10-25 20:25:42-04:00,2022-10-25 20:27:53-04:00,LINESTRING (-82.5793575000000004 29.9798012999...
40279,34471,837934,2022-10-30 01:52:42-04:00,2022-10-30 01:54:53-04:00,LINESTRING (-82.7853083600000019 30.2982544900...
40280,34471,837935,2022-10-30 02:08:36-04:00,2022-10-30 02:08:46-04:00,LINESTRING (-82.9850299999999947 30.4801399999...


In [9]:
trip_df.to_csv('trip_5.csv')

<h3>For retail stop</h3>

In [10]:
food_df = pd.read_csv('high_200m_food_related_stop.csv')
food_df

,Unnamed: 0,id,user_id,started_at,finished_at,lat,lon,retail_id,retail_lat,retail_lon
0,0,8,2,2022-09-10 02:09:19-04:00,2022-09-10 02:11:50-04:00,29.655040,-81.657860,145696,29.655165,-81.656845
1,1,16,2,2022-09-26 02:07:17-04:00,2022-09-26 02:19:51-04:00,29.655035,-81.657880,145696,29.655165,-81.656845
2,2,18,3,2022-09-05 20:01:05-04:00,2022-09-05 20:04:13-04:00,29.651755,-82.339455,70229,29.653463,-82.339195
3,3,20,3,2022-09-07 20:02:10-04:00,2022-09-07 20:06:00-04:00,29.651742,-82.339464,70229,29.653463,-82.339195
4,4,22,3,2022-09-08 20:01:10-04:00,2022-09-08 20:04:18-04:00,29.651755,-82.339455,70229,29.653463,-82.339195
...,...,...,...,...,...,...,...,...,...,...
40597,40597,311517,34461,2022-10-22 16:32:45-04:00,2022-10-22 16:40:23-04:00,30.324820,-81.397910,5905,30.324614,-81.397491
40598,40598,311526,34471,2022-10-20 07:36:51-04:00,2022-10-20 07:46:01-04:00,29.599455,-82.087014,16847,29.598845,-82.086830
40599,40599,311531,34471,2022-10-20 08:02:46-04:00,2022-10-20 08:06:32-04:00,29.265735,-82.192842,164488,29.267069,-82.192276
40600,40600,311534,34471,2022-10-22 02:50:59-04:00,2022-10-22 03:00:35-04:00,29.407710,-82.248080,18330,29.406757,-82.248161


In [11]:
food_df['started_at'] = pd.to_datetime(food_df['started_at'])
ID_list=stop_df['user_id'].unique()
len(ID_list)

10096

In [12]:
time_window = datetime.timedelta(minutes=15)

miss = 0
check_len = 0
data_to_append_list = []
for id in tqdm(ID_list):
    stop_by_user = food_df[food_df['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for stop_row in stop_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > stop_row[4] - time_window) &
            (related_tripled_by_user['finished_at'] <= stop_row[4])
        ]
        if len(tripled_found) == 0: continue #did not found related trip
        #print(tripled_found)
        #check if there if repeat data
        if len(tripled_found) > 1: 
            check_len += 1
            tripled_found = tripled_found.nlargest(1, 'finished_at')
        
        '''
            The end location and time is start time of next stop point.
        '''
        
        # data_to_append = {'deviceID': id, 'tripleg_ID': tripled_found.iloc[0]['id'], 
        #                   'trip_start_location': Point(list(wkt.loads(tripled_found.iloc[0]['geom']).coords)[0]),
        #                   'trip_end_location': (stop_row[6], stop_row[7]),
        #                   'trip_start_timestamp': tripled_found.iloc[0]['started_at'],
        #                   'trip_end_timestamp': stop_row[4]}
        data_to_append = {'user_id': id, 'tripleg_ID': tripled_found.iloc[0]['id'], 
                          'trip_started_at': tripled_found.iloc[0]['started_at'],
                          'trip_finished_at': tripled_found.iloc[0]['finished_at'],
                          'trip':tripled_found.iloc[0]['geom']}
        
        data_to_append_list.append(data_to_append)
        

food_trip_df = pd.DataFrame(data_to_append_list, columns=['user_id', 'tripleg_ID', 'trip_started_at', 'trip_finished_at', 'trip'])

100%|██████████| 10096/10096 [00:38<00:00, 262.55it/s]


In [13]:
food_trip_df

,user_id,tripleg_ID,trip_started_at,trip_finished_at,trip
0,22,1064,2022-09-06 13:42:11-04:00,2022-09-06 13:42:22-04:00,LINESTRING (-81.4465212799999989 30.3217291799...
1,22,1112,2022-09-08 22:39:14-04:00,2022-09-08 22:40:42-04:00,LINESTRING (-81.4955279999999931 30.3301033000...
2,22,1149,2022-09-10 16:54:37-04:00,2022-09-10 16:55:04-04:00,LINESTRING (-81.4905524299999939 30.3289389599...
3,22,1195,2022-09-13 16:55:10-04:00,2022-09-13 16:55:25-04:00,LINESTRING (-81.4946293800000063 30.3275656699...
4,22,1202,2022-09-13 21:16:06-04:00,2022-09-13 21:16:43-04:00,LINESTRING (-81.5225891000000047 30.3481650000...
...,...,...,...,...,...
8181,33957,836611,2022-10-27 00:34:06-04:00,2022-10-27 00:35:51-04:00,LINESTRING (-81.7667699999999940 30.1423600000...
8182,34110,836996,2022-10-29 17:55:23-04:00,2022-10-29 17:56:30-04:00,LINESTRING (-82.0392680199999944 29.1262364399...
8183,34110,836996,2022-10-29 17:55:23-04:00,2022-10-29 17:56:30-04:00,LINESTRING (-82.0392680199999944 29.1262364399...
8184,34471,837924,2022-10-20 07:58:13-04:00,2022-10-20 08:01:12-04:00,LINESTRING (-82.1521419000000037 29.2687142000...


In [14]:
print(len(food_trip_df))

8186


In [15]:
food_trip_df.to_csv('trip_food_relate_15.csv')